In [ ]:

from keras import Model, Input
from keras.layers import Conv2D, UpSampling2D, Conv2DTranspose , BatchNormalization ,Dropout ,MaxPool2D, concatenate,MaxPooling2D
from keras.activations import relu
import numpy as np


## Inception network


In [ ]:
def inception_block(prev_layer,no_of_filters,pooling):
    conva = Conv2D(filters=no_of_filters,kernel_size=(1,1),activation='relu',padding='same')(prev_layer)
    conva = BatchNormalization()(conva)

    convb = Conv2D(filters=no_of_filters,kernel_size=(1,1),activation='relu',padding='same')(prev_layer)
    convb = BatchNormalization()(convb)
    convb = Conv2D(filters=no_of_filters,kernel_size=(3,3),activation='relu',padding='same')(convb)
    convb = BatchNormalization()(convb)

    convc = Conv2D(filters=no_of_filters,kernel_size=(1,1),activation='relu',padding='same')(prev_layer)
    convc = BatchNormalization()(convc)
    convc = Conv2D(filters=no_of_filters,kernel_size=(3,3),activation='relu',padding='same')(convc)
    convc = BatchNormalization()(convc)
    convc = Conv2D(filters=no_of_filters,kernel_size=(3,3),activation='relu',padding='same')(convc)
    convc = BatchNormalization()(convc)

    max_1 = MaxPool2D(pool_size=(3,3),strides=(1,1),padding='same')(prev_layer)
    convd = Conv2D(filters=no_of_filters,kernel_size=(1,1),activation='relu',padding='same')(max_1)
    convd = BatchNormalization()(convd)

    concatenated_layers = concatenate([conva,convb,convc,convd],axis=3)

    if pooling:
        concatenated_layers = MaxPool2D(pool_size=(2,2))(concatenated_layers)

    return concatenated_layers



## Encoding and Decoding



In [ ]:
 def encoder_decoder():
        inputs = Input(shape=(256,256,3))

        conv1 = inception_block(inputs, 32, True)

        conv2 = inception_block(conv1, 64, True)
        conv3 = inception_block(conv2,128,True)
        conv4 = inception_block(conv3,256,True)

        conv5 = inception_block(conv4,512,False)
        conv6 = inception_block(conv5,512,False)

        print(conv1.shape,conv2.shape,conv3.shape,conv4.shape,conv5.shape,conv6.shape)
        up1 = concatenate([Conv2DTranspose(512,(2,2),strides=(1,1),padding='same')(conv6),conv4],axis=3)
        conv7 = inception_block(up1,256,False)


        up2 =  concatenate([Conv2DTranspose(256,(2,2),strides=(2,2),padding='same')(conv7),conv3],axis=3)
        conv8 = inception_block(up2,128,False)


        up3 =  concatenate([Conv2DTranspose(128,(2,2),strides=(2,2),padding='same')(conv8),conv2],axis=3)
        conv9 = inception_block(up3,64,False)



        up4 =  concatenate([Conv2DTranspose(64,(2,2),strides=(2,2),padding='same')(conv9),conv1],axis=3)
        conv9 = inception_block(up4,32,False)
        print(conv9.shape)

        conv10 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv9), inputs], axis=3)

        conv11 = Conv2D(32, (3, 3), activation='relu', padding='same')(conv10)

        outputs = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(conv11)

        return inputs, outputs


In [ ]:
# a = Conv2DTranspose(1024,(2,2),strides=(1,1),padding='same')(conv6)
# print (a.shape)

## Bulit a Model


In [ ]:
def bulid_model():
    inputs, outputs = encoder_decoder()
    model =Model(inputs,outputs,name="Inception_UNET")
    print(outputs.shape)
    return model



In [ ]:
# def block(prevlayer, a, b, pooling):
#     conva = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
#     conva = BatchNormalization()(conva)
#     conva = Conv2D(b, (3, 3), activation='relu', padding='same')(conva)
#     conva = BatchNormalization()(conva)
#     if True == pooling:
#         conva = MaxPooling2D(pool_size=(2, 2))(conva)


#     convb = Conv2D(a, (5, 5), activation='relu', padding='same')(prevlayer)
#     convb = BatchNormalization()(convb)
#     convb = Conv2D(b, (5, 5), activation='relu', padding='same')(convb)
#     convb = BatchNormalization()(convb)
#     if True == pooling:
#         convb = MaxPooling2D(pool_size=(2, 2))(convb)

#     convc = Conv2D(b, (1, 1), activation='relu', padding='same')(prevlayer)
#     convc = BatchNormalization()(convc)
#     if True == pooling:
#         convc = MaxPooling2D(pool_size=(2, 2))(convc)

#     convd = Conv2D(a, (3, 3), activation='relu', padding='same')(prevlayer)
#     convd = BatchNormalization()(convd)
#     convd = Conv2D(b, (1, 1), activation='relu', padding='same')(convd)
#     convd = BatchNormalization()(convd)
#     if True == pooling:
#         convd = MaxPooling2D(pool_size=(2, 2))(convd)

#     up = concatenate([conva, convb, convc, convd])
#     return up

In [ ]:
# inputs = Input((256, 256,3))

# conv1 = block(inputs, 16, 32, True)

# conv2 = block(conv1, 32, 64, True)

# conv3 = block(conv2, 64, 128, True)

# conv4 = block(conv3, 128, 256, True)

# conv5 = block(conv4, 256, 512, True)
# print(conv1.shape)
# # **** decoding ****
# xx = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
# up1 = block(xx, 512, 128, False)

# xx = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(up1), conv3], axis=3)
# up2 = block(xx, 256, 64, False)

# xx = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(up2), conv2], axis=3)
# up3 = block(xx, 128, 32, False)


# xx = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(up3), conv1], axis=3)
# up4 = block(xx, 64, 16, False)

# xx = concatenate([Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(up4), inputs], axis=3)
# print(xx.shape)

# # xx = Conv2D(32, (3, 3), activation='relu', padding='same')(xx)
# # print(xx.shape)SS



# # xx = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(xx)


# # # model = Model(inputs=[inputs], outputs=[xx])
# # print(xx.shape)

In [ ]:
import numpy as np

# Example array
x = np.array([[1, 2],
              [3, 4]])

# Before expansion
print("Shape before:", x.shape)

# Expand dimensions
x_expanded = np.expand_dims(x, axis=-1)

# After expansion
print("Shape after:", x_expanded.shape)
print(x_expanded)

Shape before: (2, 2)
Shape after: (2, 2, 1)
[[[1]
  [2]]

 [[3]
  [4]]]
